In [1]:
from pprint import pprint
import random

import sqlalchemy as db
from sqlalchemy.orm import relationship
from sqlalchemy.ext.asyncio import create_async_engine

from SharedBackend.managers import *

## 1. Overview

### 1.1 Architecture Diagram

In [2]:
# Placeholder for Mermaid diagram showing:
# - BaseSchema <|-- EntitySchema
# - GenericManager <|-- EntityManager
# - SQLAlchemy <-> Pydantic integration

### 1.2 Key Features

- Declarative ORM Model with automatic UUID generation
- Pydantic Integration with auto-generated models (Post/Patch/Put)
- Async CRUD Operations with relationship handling
- Entity Tracking System for cross-table relationships
- Advanced Query Building with nested joins/filters

## 2. Setup & Configuration



### 2.1 Database Initialization

In [3]:
DATABASE_URL = "sqlite+aiosqlite:///:memory:"
SUPPORTS_SCHEMA = False
UPSTREAM_ID = "12345678-unset-upid"


def get_engine():
    if SUPPORTS_SCHEMA:
        engine = create_async_engine(DATABASE_URL, execution_options={"schema_translate_map": {None: "sharedbackend"}})
    else:
        engine = create_async_engine(DATABASE_URL)
    return engine

In [4]:
async def setup(_engine):
    async with _engine.begin() as conn:
        if SUPPORTS_SCHEMA: await conn.execute(db.text(f"CREATE SCHEMA IF NOT EXISTS sharedbackend"))
        await conn.run_sync(BaseSchema.metadata.create_all)

## 3. Basic CRUD Operations

In [5]:
class UserSchema(BaseSchema):
    __tablename__ = "users"

    name = db.Column(db.String(50))
    email = db.Column(db.String(100), unique=True)
    phone = db.Column(db.String(20))


class UserManager(GenericManager[UserSchema]):
    pass

In [6]:
engine = get_engine()
user_manager = UserManager(engine)

await setup(engine)

### 3.1 Create Operations

In [7]:
async with user_manager.session_factory() as session:
    user = await user_manager.create(
        UserSchema(name="John Doe", email="john@test.com"),
        session=session
    )
    user.phone = "+910000000001"
    await session.commit()
user.model_dump()

{'name': 'John Doe',
 'email': 'john@test.com',
 'phone': '+910000000001',
 'uid': 'users_90f5a5c4-895c-43ff-b4c3-33b5d4f78a03',
 'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'updated_at': None,
 'deleted_at': None}

In [8]:
async with user_manager.session_factory() as session:
    user = await user_manager.create(
        UserSchema(name="Smith Doe", email="smith@test.com", phone="+910000000002"),
        session=session,
        upstreamId=UPSTREAM_ID,
    )
    entity = await user_manager.entity_manager.fetch_one(filters={"upstreamId": UPSTREAM_ID}, session=session)
    await session.commit()
pprint(user.model_dump())
pprint(entity.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'smith@test.com',
 'name': 'Smith Doe',
 'phone': '+910000000002',
 'uid': 'users_08dba04d-ac86-4fe2-b372-719b20cbf34a',
 'updated_at': None}
{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'downstreamId': 'users_08dba04d-ac86-4fe2-b372-719b20cbf34a',
 'tablename': 'users',
 'uid': 'entities_1687f02b-2577-426d-a82b-dc539cc90df0',
 'updated_at': None,
 'upstreamId': '12345678-unset-upid'}


### 3.2 Read Operations

In [9]:
user = await user_manager.fetch(user.uid)
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'smith@test.com',
 'name': 'Smith Doe',
 'phone': '+910000000002',
 'uid': 'users_08dba04d-ac86-4fe2-b372-719b20cbf34a',
 'updated_at': None}


In [10]:
user = await user_manager.fetch_one(filters={"email": "john@test.com"})
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'john@test.com',
 'name': 'John Doe',
 'phone': '+910000000001',
 'uid': 'users_90f5a5c4-895c-43ff-b4c3-33b5d4f78a03',
 'updated_at': datetime.datetime(2025, 3, 6, 8, 46, 3)}


In [11]:
users = await user_manager.fetch_all(sorts=["+name"])
pprint(users.model_dump())

{'count': 2,
 'items': [{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
            'deleted_at': None,
            'email': 'smith@test.com',
            'name': 'Smith Doe',
            'phone': '+910000000002',
            'uid': 'users_08dba04d-ac86-4fe2-b372-719b20cbf34a',
            'updated_at': None},
           {'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
            'deleted_at': None,
            'email': 'john@test.com',
            'name': 'John Doe',
            'phone': '+910000000001',
            'uid': 'users_90f5a5c4-895c-43ff-b4c3-33b5d4f78a03',
            'updated_at': datetime.datetime(2025, 3, 6, 8, 46, 3)}]}


### 3.3 Update Operations

In [12]:
async with user_manager.session_factory() as session:
    user = await user_manager.update(
        user.uid,
        {"name": "John Smith"},
        session=session
    )
    await session.commit()
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'john@test.com',
 'name': 'John Smith',
 'phone': '+910000000001',
 'uid': 'users_90f5a5c4-895c-43ff-b4c3-33b5d4f78a03',
 'updated_at': None}


In [13]:
async with user_manager.session_factory() as session:
    user = await user_manager.fetch(
        user.uid,
        session=session
    )
    user.email = "johnsmith@test.com"
    await session.commit()
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'johnsmith@test.com',
 'name': 'John Smith',
 'phone': '+910000000001',
 'uid': 'users_90f5a5c4-895c-43ff-b4c3-33b5d4f78a03',
 'updated_at': None}


In [14]:
async with user_manager.session_factory() as session:
    user = await user_manager.update_one(
        filters={"email": "johnsmith@test.com"},
        updates={"name": "Mr Smith"},
        session=session,
    )
    await session.commit()
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'johnsmith@test.com',
 'name': 'Mr Smith',
 'phone': '+910000000001',
 'uid': 'users_90f5a5c4-895c-43ff-b4c3-33b5d4f78a03',
 'updated_at': None}


### 3.4 Delete Operations

In [15]:
user = await user_manager.delete_one(filters={"email": "smith@test.com"})
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'smith@test.com',
 'name': 'Smith Doe',
 'phone': '+910000000002',
 'uid': 'users_08dba04d-ac86-4fe2-b372-719b20cbf34a',
 'updated_at': None}


### 3.5 Create or Fetch

In [16]:
user = await user_manager.create_or_fetch(
    UserSchema(name="John Doe", email="john@test.com", phone="+910000000003"),
    session=session
)
pprint(user.model_dump())
user = await user_manager.create_or_fetch(
    UserSchema(name="John Doe", email="john@test.com"),
    session=session
)
pprint(user.model_dump())

{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'john@test.com',
 'name': 'John Doe',
 'phone': '+910000000003',
 'uid': 'users_ee8ba45c-8536-497e-a7da-c1c3e4988bbd',
 'updated_at': None}
{'created_at': datetime.datetime(2025, 3, 6, 8, 46, 3),
 'deleted_at': None,
 'email': 'john@test.com',
 'name': 'John Doe',
 'phone': '+910000000003',
 'uid': 'users_ee8ba45c-8536-497e-a7da-c1c3e4988bbd',
 'updated_at': None}


## 4. Advanced Features

In [17]:
class AddressSchema(BaseSchema):
    __tablename__ = "addresses"

    user_uid = db.Column(db.String, db.ForeignKey("users2.uid"))
    type = db.Column(db.Enum("home", "work", "other"), nullable=False)
    line1 = db.Column(db.String(100))
    line2 = db.Column(db.String(100))
    city = db.Column(db.String(50))
    state = db.Column(db.String(50))
    country = db.Column(db.String(50))
    postal_code = db.Column(db.String(20))


class BankSchema(BaseSchema):
    __tablename__ = "banks"

    user_uid = db.Column(db.String, db.ForeignKey("users2.uid"))
    name = db.Column(db.String(100))
    account_number = db.Column(db.INT, unique=True)
    ifsc_code = db.Column(db.String(20))


class UserSchema(BaseSchema):
    __tablename__ = "users2"

    name = db.Column(db.String(50))
    email = db.Column(db.String(100), unique=True)
    phone = db.Column(db.String(20))

    addresses = relationship(
        "AddressSchema",
        uselist=True,
        backref="user",
        cascade="all, delete-orphan",
    )

    banks = relationship(
        "BankSchema",
        uselist=True,
        backref="user",
        cascade="all, delete-orphan",
    )


class AddressManager(GenericManager[AddressSchema]):
    pass


class BankManager(GenericManager[BankSchema]):
    pass


class UserManager(GenericManager[UserSchema]):
    def __init__(self, engine):
        super().__init__(engine)
        self.address_manager = AddressManager(engine)
        self.bank_manager = BankManager(engine)

/var/folders/q2/wwrtn_qd3xd4jwcthb0jj9l80000gn/T/ipykernel_80799/3846552836.py:23: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.UserSchema, and will be replaced in the string-lookup table.
  class UserSchema(BaseSchema):


In [18]:
engine = get_engine()
user_manager = UserManager(engine)

await setup(engine)

In [19]:
async with user_manager.session_factory() as session:
    users = await user_manager.create_all([
        UserSchema(
            name=f"John Doe the {i + 1}",
            email=f"johnthepepe{i + 1}@test.com",
            phone=f"+91000000000{i + 1}",
        ) for i in range(5)
    ], session=session, joins=[UserSchema.addresses, UserSchema.banks])
    for i, user in enumerate(users.items):
        a = await user_manager.address_manager.create(AddressSchema(
            type=random.choice(["home", "work", "other"]),
            line1=f"House No {i + 1}",
            city="Bangalore",
            state="Karnataka",
            country="India",
            postal_code="560001",
        ), session=session)
        user.addresses.append(a)
        b = await user_manager.bank_manager.create(BankSchema(
            name="ICICI Bank",
            account_number=random.randint(1000000000, 9999999999),
            ifsc_code=f"ICIC000000{i + 1}",
        ), session=session)
        user.banks.append(b)
    await session.commit()

### 4.1 Nested Filters

In [20]:
users = await user_manager.fetch_all(filters={"addresses": {"type": "home"}}, joins=[UserSchema.addresses])
pprint(users.model_dump())

{'count': 3,
 'items': [{'addresses': [{'city': 'Bangalore',
                           'country': 'India',
                           'created_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
                           'deleted_at': None,
                           'line1': 'House No 2',
                           'line2': None,
                           'postal_code': '560001',
                           'state': 'Karnataka',
                           'type': 'home',
                           'uid': 'addresses_6e0723fe-d216-4307-aef0-e02a175d36cf',
                           'updated_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
                           'user_uid': 'users2_48e14ee4-e751-4043-906a-6f46f1af49f0'}],
            'banks': None,
            'created_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
            'deleted_at': None,
            'email': 'johnthepepe3@test.com',
            'name': 'John Doe the 3',
            'phone': '+910000000003',
            'uid': 'users

### 4.2 Nested Joins

In [21]:
user = await user_manager.address_manager.fetch_one(joins=[[AddressSchema.user, UserSchema.banks]])
pprint(user.model_dump())

{'city': 'Bangalore',
 'country': 'India',
 'created_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
 'deleted_at': None,
 'line1': 'House No 1',
 'line2': None,
 'postal_code': '560001',
 'state': 'Karnataka',
 'type': 'other',
 'uid': 'addresses_6cb04b0f-ce64-4d46-a834-fd0a02fb6fef',
 'updated_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
 'user': {'banks': [{'account_number': 7411396873,
                     'created_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
                     'deleted_at': None,
                     'ifsc_code': 'ICIC0000001',
                     'name': 'ICICI Bank',
                     'uid': 'banks_a512fe97-d9f0-4d4b-bc41-37e26cfccde4',
                     'updated_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
                     'user_uid': 'users2_40bdc524-f2a9-4f1c-85d9-8acde2f0f482'}],
          'created_at': datetime.datetime(2025, 3, 6, 8, 46, 4),
          'deleted_at': None,
          'email': 'johnthepepe5@test.com',
          'name': 'John Doe 

## 5. Testing